In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [105]:
classes = [
    'blackheads',
    'inflamed_acne',
    'large_pimples'
] 
train_dir = "./data2/train/"
valid_dir = "./data2/valid/"
test_dir = "./data2/test/"

In [97]:
def read_annotations(annotation_file, num):
    image_paths = []
    combined_labels = []
    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            if num == 0:
                image_paths.append(valid_dir + parts[0])# Full image path
            else:
                image_paths.append(train_dir + parts[0])# Full image path
            # Format: [x_min, y_min, x_max, y_max, class_label]
            combined_labels.append(list(map(int, parts[1].split(','))))
    return image_paths, combined_labels


In [98]:
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    # Normalize pixel values to range [0, 1]
    image /= 255.0
    return image

In [99]:
train_image_paths, train_combined_labels = read_annotations("data2/train/_annotations.txt", 1)
valid_image_paths, valid_combined_labels = read_annotations("data2/valid/_annotations.txt", 0)

In [100]:
X_train = np.array([preprocess_image(image_path) for image_path in train_image_paths])
X_valid = np.array([preprocess_image(image_path) for image_path in valid_image_paths])
y_train = np.array(train_combined_labels)
y_valid = np.array(valid_combined_labels)

In [101]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5)  # Output layer with 5 units for class label and bounding box coordinates
])

c:\Users\gs2me\OneDrive\Desktop\Projects\burn-scan-ai\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [102]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [113]:
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 349ms/step - accuracy: 0.6466 - loss: 8899.6250 - val_accuracy: 0.7073 - val_loss: 9430.1484


In [114]:
def read_test_annotations(annotation_file):
    image_paths = []
    combined_labels = []
    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split(" ")
            image_paths.append(os.path.join(test_dir, parts[0]))  # Full image path
            # Format: [x_min, y_min, x_max, y_max, class_label]
            combined_labels.append(list(map(int, parts[1].split(','))))
    return image_paths, combined_labels

# Read test annotations
test_annotation_file = os.path.join(test_dir, "_annotations.txt")
test_image_paths, test_combined_labels = read_test_annotations(test_annotation_file)

# Preprocess test images
X_test = np.array([preprocess_image(image_path) for image_path in test_image_paths])
y_test = np.array(test_combined_labels)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.5000 - loss: 10119.9492
Test Loss: 10119.94921875
Test Accuracy: 0.5


In [117]:
model.save("model.keras")
